# Metadata Refresh for SQL Server


## Insert Server Name (if needed)

> The 'Server Name' is a **Friendly** name. It does not need to be the same as the true Server name (which is determined by the 'Connection' selected at runtime)

In [ ]:
DECLARE @server_name NVARCHAR(128) = /*${Metadata_Server_Name}*/'dev'/**/

IF NOT EXISTS (SELECT 1 FROM [meta].[dim_server] WHERE [server_name] = @server_name)
	BEGIN
		INSERT INTO [meta].[dim_server]
				   ([server_name]
				   ,[server_type]
				   )
			 VALUES
				   ( @server_name
				   , 'SQL Server')
	END;

SELECT @@ROWCOUNT AS [row_count];


## Truncate staging tables

In [ ]:
TRUNCATE TABLE [stg].[dim_database];
TRUNCATE TABLE [stg].[dim_table];
TRUNCATE TABLE [stg].[dim_column];

## Retrieve all Databases

In [ ]:
DECLARE @server_name NVARCHAR(128) = /*${Metadata_Server_Name}*/'dev'/**/

SELECT @server_name AS server_name
	 , d.[name] AS [database_name]
	 , CAST(d.[create_date] AS DATE) AS [database_create_date]
	 , CASE WHEN t.[database_id] IS NULL
		    THEN 'N'
			ELSE 'Y'
		END AS [change_tracking_enabled]
	 , 'Metadata Refresh' AS [row_data_source]
  FROM sys.databases d
  LEFT JOIN sys.change_tracking_databases t
    ON t.[database_id] = d.[database_id];

## Merge stg to meta schema - dim_database

In [ ]:
DELETE [meta].[dim_database]
  FROM [meta].[dim_database] AS e
  LEFT JOIN [stg].[dim_database] AS s
    ON s.[server_name] = e.[server_name]
   AND s.[database_name] = e.[database_name]
 WHERE e.[server_name] = /*${Metadata_Server_Name}*/'dev'/**/
   AND s.[database_name] IS NULL;

INSERT [meta].[dim_database]
SELECT s.[server_name]
     , s.[database_name]
     , s.[database_create_date]
     , s.[change_tracking_enabled]
     , 'N'
     , 'N'
  FROM [stg].[dim_database] AS s
  LEFT JOIN [meta].[dim_database] AS e
    ON e.[server_name] = s.[server_name]
   AND e.[database_name] = s.[database_name]
 WHERE s.[server_name] = /*${Metadata_Server_Name}*/'dev'/**/
   AND e.[database_name] IS NULL;

SELECT @@ROWCOUNT AS [row_count];

## src to stg - SQL Server tables

In [ ]:
--USE /*${SQL_Server_Database}*/[eltsnap_v2]/**/;

DECLARE @Metadata_Server_Name SYSNAME = /*${Metadata_Server_Name}*/'dev'/**/;

WITH [identity_column] AS
(
SELECT s.[name] AS [TABLE_SCHEMA]
	 , o.[name] AS [TABLE_NAME]
  FROM sys.objects o 
  JOIN sys.columns c
    ON c.[object_id] = o.[object_id]
  JOIN sys.schemas s
    on s.[schema_id] = o.[schema_id]
 WHERE c.is_identity = 1
)
, [primary_key] AS
(
SELECT [TABLE_SCHEMA]
	 , [TABLE_NAME]
  FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS
 WHERE CONSTRAINT_TYPE = 'PRIMARY KEY'
)
SELECT DISTINCT @Metadata_Server_Name AS [server_name]
     , t.[TABLE_CATALOG] AS [database_name] 
     , t.[TABLE_SCHEMA] AS [table_schema] 
     , t.[TABLE_NAME] AS [table_name] 
     , CAST(t.[TABLE_TYPE] AS NVARCHAR(32)) AS [table_type] 
     , CASE WHEN i.[TABLE_NAME] IS NULL THEN N'N'
            ELSE N'Y'
       END AS [has_identity]
     , CASE WHEN p.[TABLE_NAME] IS NULL THEN N'N'
            ELSE N'Y'
       END AS [has_primary_key]
     , N'SQL' AS [row_data_source] 
  FROM INFORMATION_SCHEMA.TABLES t
  LEFT JOIN [identity_column] i
    ON i.[TABLE_SCHEMA] = t.[TABLE_SCHEMA]
   AND i.[TABLE_NAME] = t.[TABLE_NAME]
  LEFT JOIN [primary_key] p
    ON p.[TABLE_SCHEMA] = t.[TABLE_SCHEMA]
   AND p.[TABLE_NAME] = t.[TABLE_NAME]
 WHERE [TABLE_TYPE] IN ('BASE TABLE', 'VIEW');

## src to stg - SQL Server columns

In [ ]:
USE /*${SQL_Server_Database}*/[eltsnap_v2]/**/;

DECLARE @Metadata_Server_Name SYSNAME = /*${Metadata_Server_Name}*/'dev'/**/;

WITH [pk_columns] AS
(
SELECT tc.[TABLE_CATALOG] AS [database_name]
	 , tc.[TABLE_SCHEMA] AS [table_schema]
	 , tc.[TABLE_NAME] AS [table_name]
	 , cc.[COLUMN_NAME] AS [column_name]
  FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
  JOIN INFORMATION_SCHEMA.CONSTRAINT_COLUMN_USAGE cc 
		ON tc.CONSTRAINT_NAME = cc.Constraint_name
    WHERE tc.CONSTRAINT_TYPE = 'Primary Key'
)
SELECT DISTINCT @Metadata_Server_Name AS [server_name]
	 , c.[TABLE_CATALOG] AS [database_name] 
	 , c.[TABLE_SCHEMA] AS [table_schema] 
	 , c.[TABLE_NAME] AS [table_name] 
	 , c.[COLUMN_NAME] AS [column_name] 
	 , c.[ORDINAL_POSITION] AS [ordinal_position] 
	 , CAST(c.[IS_NULLABLE] AS NVARCHAR(3)) AS [is_nullable] 
	 , c.[DATA_TYPE] AS [data_type] 
	 , c.[CHARACTER_MAXIMUM_LENGTH] AS [char_max_length] 
	 , CASE WHEN p.[table_name] IS NULL
		    THEN N'N'
			ELSE N'Y'
		END AS [primary_key]
	 , N'SQL' AS [row_data_source]
FROM INFORMATION_SCHEMA.COLUMNS c 
  LEFT JOIN [pk_columns] p
    ON p.[database_name] = c.[TABLE_CATALOG]
   AND p.[table_schema] = c.[TABLE_SCHEMA]
   AND p.[table_name] = c.[TABLE_NAME]
   AND p.[column_name] = c.[COLUMN_NAME];

## Merge - stg to meta - dim_table

In [ ]:
MERGE [meta].[dim_table] AS DST
USING [stg].[dim_table] AS SRC
   ON src.[server_name] = dst.[server_name]
   AND src.[database_name] = dst.[database_name]
   AND src.[table_schema] = dst.[table_schema]
   AND src.[table_name] = dst.[table_name]

WHEN MATCHED AND (    DST.[table_type] != SRC.[table_type] 
                   OR DST.[has_identity] != SRC.[has_identity] 
	           OR DST.[has_primary_key] != SRC.[has_primary_key] ) THEN
		UPDATE SET [table_type] =  SRC.[table_type] 
                 , [has_identity] =  SRC.[has_identity] 
                 , [has_primary_key] = SRC.[has_primary_key] 

WHEN NOT MATCHED THEN 
INSERT ( [server_name]
      ,[database_name]
      ,[table_schema]
      ,[table_name]
      ,[table_type]
      ,[has_identity]
      ,[has_primary_key]
      ,[enable_change_tracking]
	   )
VALUES (
	   [server_name]
      ,[database_name]
      ,[table_schema]
      ,[table_name]
      ,[table_type]
      ,[has_identity]
      ,[has_primary_key]
      ,'N'	
	   )
WHEN NOT MATCHED BY SOURCE AND DST.[server_name] = /*${Metadata_Server_Name}*/'dev'/**/ THEN
	 DELETE;

SELECT @@ROWCOUNT AS [row_count];

## Merge - stg to meta - dim_column

In [ ]:
MERGE [meta].[dim_column] AS DST
USING [stg].[dim_column] AS SRC
   ON src.[server_name] = dst.[server_name]
   AND src.[database_name] = dst.[database_name]
   AND src.[table_schema] = dst.[table_schema]
   AND src.[table_name] = dst.[table_name]
   AND src.[column_name] = dst.[column_name]

WHEN MATCHED AND (    DST.[ordinal_position] != SRC.[ordinal_position] 
                   OR DST.[is_nullable] != SRC.[is_nullable] 
		   OR DST.[data_type] != SRC.[data_type]
                   OR DST.[char_max_length] != SRC.[char_max_length] 
		   OR DST.[primary_key] != SRC.[primary_key] 
               ) THEN
UPDATE SET [ordinal_position] = SRC.[ordinal_position] 
         , [is_nullable] = SRC.[is_nullable] 
         , [data_type] = SRC.[data_type] 
         , [char_max_length] = SRC.[char_max_length]
		   , [primary_key] = SRC.[primary_key]

WHEN NOT MATCHED THEN 
INSERT (  [server_name]
	    , [database_name]
        , [table_schema]
        , [table_name]
        , [column_name]
        , [ordinal_position]
        , [is_nullable]
        , [data_type]
        , [char_max_length]
		, [primary_key]
	   )
VALUES (
		  [server_name]
	    , [database_name]
        , [table_schema]
        , [table_name]
        , [column_name]
        , [ordinal_position]
        , [is_nullable]
        , [data_type]
        , [char_max_length]
		, [primary_key]
	   )
WHEN NOT MATCHED BY SOURCE AND DST.[merge_type] = 't1' AND DST.[server_name] = /*${Metadata_Server_Name}*/'dev'/**/ THEN
	 DELETE;

SELECT @@ROWCOUNT AS [row_count];

## Merge stg to dbo schema - dim_table

In [ ]:
EXEC [elt].[build_and_execute_merge_for_dimension] 'stg', 'dim_table', 'dbo', 'dim_table', 'Standard', 'server_name,database_name,table_schema,table_name','table_type';

## Merge stg to dbo schema - dim_column

In [ ]:
EXEC [elt].[build_and_execute_merge_for_dimension] 'stg', 'dim_column', 'dbo', 'dim_column', 'Standard', 'server_name,database_name,table_schema,table_name,column_name','is_nullable,data_type,char_max_length';

## dim delete - dbo - dim_table

In [ ]:
WITH [deleted_rows] AS
(
SELECT [server_name]
	 , [database_name]
	 , [table_schema]
	 , [table_name]
  FROM [dbo].[dim_table]
EXCEPT 
SELECT [server_name]
	 , [database_name]
	 , [table_schema]
	 , [table_name]
  FROM [meta].[dim_table]
)
UPDATE [dbo].[dim_table]
   SET [row_expiration_date] = CAST(GETDATE() AS DATE)
     , [row_update_date] = CAST(GETDATE() AS DATE)
  FROM [dbo].[dim_table] t
  JOIN [deleted_rows] d
    ON d.[server_name]   = t.[server_name]
   AND d.[database_name] = t.[database_name]
   AND d.[table_schema]  = t.[table_schema]
   AND d.[table_name]    = t.[table_name]
 WHERE t.[row_expiration_date] = '9999-12-31';

SELECT @@ROWCOUNT AS [row_count];

## dim delete - dbo - dim_column

In [ ]:
WITH [deleted_rows] AS
(
SELECT [server_name]
	 , [database_name]
	 , [table_schema]
	 , [table_name]
	 , [column_name]
  FROM [dbo].[dim_column]
EXCEPT 
SELECT [server_name]
	 , [database_name]
	 , [table_schema]
	 , [table_name]
	 , [column_name]
  FROM [meta].[dim_column]
)
UPDATE [dbo].[dim_column]
   SET [row_expiration_date] = CAST(GETDATE() AS DATE)
     , [row_update_date] = CAST(GETDATE() AS DATE)
  FROM [dbo].[dim_column] c
  JOIN [deleted_rows] d
    ON d.[server_name]   = c.[server_name]
   AND d.[database_name] = c.[database_name]
   AND d.[table_schema]  = c.[table_schema]
   AND d.[table_name]    = c.[table_name]
   AND d.[column_name]   = c.[column_name]
 WHERE c.[row_expiration_date] = '9999-12-31';

SELECT @@ROWCOUNT AS [row_count];